1. Use nike website, to get all the product details for all products on sale.
2. The Nike website new products load up as we keep scrolling down until all the 1000+ products are loaded up.
    This is called infinite scrolling.
3. Our project, we will get past infinite scrolling to get the product details of all the 1000+ salw products.
4. For this, we will use the combination of Selenium, and BeautifulSoup.
    Selenium: for infinite scrolling
    BeautifulSoup: to get the product details.


In [1]:
# Import Libraries
#BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd

#Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains # import library to scroll to specific element 
import time


In [2]:
# Get the chrome driver and put it in the variable driver

driver = webdriver.Chrome('/Users/zachariah/Downloads/chromedriver')

/var/folders/8p/ng4nv8tj7yj2m_xtn234qv_h0000gn/T/ipykernel_6674/3430411186.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/zachariah/Downloads/chromedriver')


In [3]:
# get the url of Nike sale page

driver.get('https://www.nike.com/w/sale-3yaep')

In [4]:
# First we nedd to deal with infinite scrolling

# we will use the self scrolling code 
 
last_height = driver.execute_script('return document.body.scrollHeight')  #to get the height of the page it is at right now
last_height

5114

In [5]:
# universal Code to self scroll to the very last page of the items.

while True:
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(2)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height
    

In [6]:
# to get the html corresponding to each product
# frist we need to get the html of the page using BeautifulSoup 
# since we are using selenium we don't have to use the request library, instead we use the following code

soup = BeautifulSoup(driver.page_source, 'lxml')

In [7]:
# to get the product card of the products on the page

product_card = soup.find_all('div', class_ = 'product-card__body')


In [8]:
# lets see if we got all the product cards
len(product_card)

1128

In [9]:
# create a dataframe

df = pd.DataFrame({'Name':[''],'Sub Title':[''], 'Link':[''],'Sale Price':[''], 'Original Price':[''], 'Colors':['']})

In [10]:
# Now lets create a for loop to go through each one

for product in product_card:
    try:
        link = product.find('a', class_ = 'product-card__link-overlay').get('href')
        name = product.find('a', class_ = 'product-card__link-overlay').text
        sub_title = product.find('div', class_ = 'product-card__subtitle').text
        sale_price = product.find('div',class_ ='product-price is--current-price css-1ydfahe').text
        full_price = product.find('div',class_ ='product-price is--striked-out css-0').text
        colors = product.find('div', class_ ='product-card__product-count font-override__body1').text
        df = df.append({'Name':name,'Sub Title':sub_title, 'Link':link,'Sale Price':sale_price, 'Original Price':full_price, 'Colors':colors}, ignore_index = True)
    except:
        break

/var/folders/8p/ng4nv8tj7yj2m_xtn234qv_h0000gn/T/ipykernel_6674/368418988.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name':name,'Sub Title':sub_title, 'Link':link,'Sale Price':sale_price, 'Original Price':full_price, 'Colors':colors}, ignore_index = True)
/var/folders/8p/ng4nv8tj7yj2m_xtn234qv_h0000gn/T/ipykernel_6674/368418988.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name':name,'Sub Title':sub_title, 'Link':link,'Sale Price':sale_price, 'Original Price':full_price, 'Colors':colors}, ignore_index = True)
/var/folders/8p/ng4nv8tj7yj2m_xtn234qv_h0000gn/T/ipykernel_6674/368418988.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name':name,'Sub Title':

In [11]:
df.shape

(1129, 6)

In [12]:
df.head()

,Name,Sub Title,Link,Sale Price,Original Price,Colors
0,,,,,,
1,Nike Metcon 7,Women's Training Shoes,https://www.nike.com/t/metcon-7-womens-trainin...,$117.97,$130,3 Colors
2,Nike Air Zoom Pegasus 38,Women's Road Running Shoes,https://www.nike.com/t/air-zoom-pegasus-38-wom...,$102.97,$120,11 Colors
3,Nike Dri-FIT One Luxe,Women's Twist Standard Fit Short-Sleeve Top,https://www.nike.com/t/dri-fit-one-luxe-womens...,$40.97,$50,13 Colors
4,Nike Sportswear Essential,Women's Mid-Rise Bike Shorts,https://www.nike.com/t/sportswear-essential-wo...,$25.97,$35,9 Colors


In [13]:
# save dataframe to csv file

df.to_csv('/Users/zachariah/Downloads/nike.csv')